In [ ]:
import numpy as np

def getW(K,d):
    w = np.random.randn(K,d,d)
    return w

def getb(K,d):
    b = np.random.randn(K,d,1)
    return b

def getomega(d):
    omega = np.random.randn(d,1)
    return omega

def getmu():
    mu = np.random.randn(1)
    return mu

def getdJdomega(Z, ypsilon, c, omega, mu, K, eta_div):
    dJdOmega = Z[K-1] @ ((ypsilon - c) * eta_div( np.transpose(Z[K-1]) @ omega + mu))
    return dJdOmega

def getdJdW(P, Z, W, b, K, h, sigma_div):
    dJdW = np.zeros_like(W)
    for k in range(K):
        dJdW[k] = h*(P[k+1]*sigma_div(W[k]@Z[k]+b[k]))@np.transpose(Z[k])
    return dJdW

def getdJdb(P, Z, W, b, K, h, sigma_div):
    dJdb = np.zeros_like(b)
    dJdb = np.zeros_like(b)
    for k in range(K):
        dJdb[k] = h*(P[k+1]*sigma_div(W[k]@Z[k]+b[k]))@np.transpose(np.array([[1]*len(Z[0,0,:])]))
    return dJdb


def getYpsilon(Z, omega, mu, eta):
    return eta(np.transpose(Z[K-1]) @ omega + mu)

In [ ]:
def getZ(Y0, W, b, K, d, I, h, sigma):
    #initialize Z, Z0=Y0
    Z = np.zeros((K,d,I))
    Z[0] = Y0

    #finds Zk
    for k in range(1,K):
        u = W[k-1]@Z[k-1] + b[k-1]
        Z[k] = Z[k-1] + h*sigma(u)

    return Z

def getP(Z, ypsilon, c, omega, mu, W, b, K, d, I, h, sigma_div, eta_div):
    #initialize P
    P = np.zeros((K+1,d,I))

    #finds P_K
    u = np.transpose(Z[-1])@omega + np.full((d,I),mu)
    P[-1]= np.outer(omega,np.transpose((ypsilon-c)*eta_div(u)))

    #finds P_K-1 to P_0
    for k in range(K,-1,-1):
        s = W[k]@Z[k] + b[k]
        P[k]=P[k+1] + h*np.transpose(W[k])@(sigma_div(s)*P[k+1])

    return P

def getdJdmu(Z, ypsilon, c, omega, mu, K, eta_div):
    u = np.transpose(Z[K-1])@omega + np.full(omega.shape,mu)

    return np.transpose(eta_div(u))*(ypsilon-c)


In [ ]:
a = np.array([[1,2,3],[4,5,6]])
b = np.array([4,5,6])
#print(a@b)
#print(np.transpose(a)@b)
#print(a@np.transpose(b))
#print(np.outer(a,np.transpose(b)))
"""temp = np.zeros((d,I))
        for i in range(d0):
            temp[i] = Y0[i]
        Y0 = temp"""

print(a)
a.resize(4,3)
print(a)

In [ ]:
def trainingAlgorithm(K, d, h, tau, Y0, c, eta, sigma, eta_div, sigma_div, N = 1000):
    d0 = Y0.shape[0]
    I = Y0.shape[1]
    if d0 < d:
        Y0.resize(d,I)
    omega = getomega(d)
    mu = getmu()
    W = getW(K,d)
    b = getb(K,d)
    for i in range(N): #Number of trainingseries
        Z = getZ(Y0, W, b, K, d, I, h, sigma)
        ypsilon = getYpsilon(Z, omega, mu, eta)
        P = getP(Z, ypsilon, c, omega, mu, W, b, K, d, I, h, sigma_div, eta_div)
        
        dJdmu = getdJdmu(Z, ypsilon, c, omega, mu, K, eta_div)
        dJdomega = getdJdomega(Z, ypsilon, c, omega, mu, K, eta_div)
        dJdW = getdJdW(P, Z, W, b, K, h, sigma_div)
        dJdb = getdJdb(P, Z, W, b, K, h, sigma_div)
        
        mu = mu - tau*dJdmu
        omega = omega - tau*dJdomega
        W = W - tau*dJdW
        b = b - tau*dJdb
    return mu, omega, W, b

In [ ]:
def eta(x):
    return x

def eta_div(x):
    return 1

def sigma(x):
    return np.tanh(x)

def sigma_div(x):
    return 1/(np.cosh(x))^2

In [ ]:
def F(y):
    return 1/2*y^2

Y0 = np.linspace(-2,2,100)
c = F(Y0)



trainingAlgorithm(K, d, h, tau, Y0, c, eta, sigma, eta_div, sigma_div)